# 1. NumPy a práce se soubory

## 1.1 Import knihovny

Pro práci s NumPy použijeme standardní import pod aliasem `np`.


In [ ]:
import numpy as np


## 1.2 Textové soubory (ASCII/CSV)

Textové soubory s numerickými daty jsou běžné, ale často nepraktické. Pro načítání těchto dat v NumPy se nejčastěji používají `genfromtxt` a `loadtxt`.


Pomocí `%%file` vytvoříme soubor `ascii_data_1.txt`.


In [ ]:
%%file ascii_data_1.txt
1 -6.1 -6.1 -6.1 1
2 -15.4 -15.4 -15.4 1
3 -15.0 -15.0 -15.0 1
4 -19.3 -19.3 -19.3 1
5 -16.8 -16.8 -16.8 1
6 -11.4 -11.4 -11.4 1
7 -7.6 -7.6 -7.6 1
8 -7.1 -7.1 -7.1 1
9 -10.1 -10.1 -10.1 1
10 -9.5 -9.5 -9.5 1


Nyní soubor načteme pomocí `genfromtxt`.


In [ ]:
data = np.genfromtxt('ascii_data_1.txt')
print(data)


Stejný soubor umí načíst i `loadtxt`:


In [ ]:
data = np.loadtxt('ascii_data_1.txt')
print(data)

`savetxt` použijeme pro uložení textových dat.

- Parametr `fmt` určuje formát zápisu.
- Například `fmt="%.2f"` uloží hodnoty se dvěma desetinnými místy.
- Například `fmt="%6g"` uloží čísla v poli o minimální šířce 6 znaků.


In [ ]:
np.savetxt("ascii_data_1_new.txt", np.random.rand(5,3), fmt="%.2f")
np.savetxt("ascii_data_2_new.txt", data, fmt="%6g")


Obsah souboru můžeme vypsat:


In [ ]:
%less ascii_data_1_new.txt


In [ ]:
%less ascii_data_2_new.txt


Textové formáty (včetně CSV) jsou pro větší numerická data často nevhodné: bývají objemné, pomalejší a jejich interpretace nemusí být jednoznačná. Výhoda je hlavně snadná čitelnost v textovém editoru.


## 1.3 Binární formát NPY

Pro numerická data jsou obvykle vhodnější binární formáty: jsou menší a jednoznačněji definované.

NumPy má vlastní formát NPY. V prostředí Pythonu se používá velmi jednoduše přes `save` a `load`.


In [ ]:
np.save("binary_data_1_new.npy", np.random.rand(5,3))


In [ ]:
data = np.load("binary_data_1_new.npy")
print(data)

In [ ]:
# porovnání velikosti textového a binárního souboru
data = np.random.rand(100, 100)
np.save('binary_data_1_new.npy', data)
np.savetxt('ascii_data_1_new.txt', data)
%ls -l


## 1.4 HDF5 formát

Rozšířený standard pro ukládání numerických dat je [HDF5](https://www.hdfgroup.org/solutions/hdf5/). V Pythonu se pro něj běžně používá knihovna [h5py](https://www.h5py.org/).


In [ ]:
# pokud nemáte h5py nainstalované, můžete instalovat přímo z notebooku
# %conda install h5py
# nebo v pip prostředí
# %pip install h5py


In [ ]:
# můžete instalovat přímo z notebooku, stačí odstranit komentář
# !pip install h5py -U

In [ ]:
import h5py


V HDF5 souborech jsou data ve stromové struktuře (obdoba adresářů a souborů). Soubor se dvěma datasety můžeme vytvořit například takto:


In [ ]:
data_pole = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.int32)
data_binary = np.array([True, False, True, False, True, False], dtype=bool)
with h5py.File("test_hdf5.h5", "w") as hdf5_file:
    hdf5_file.create_dataset("data1", data=data_pole)
    hdf5_file.create_dataset("data_nahodne", data=np.random.random_sample((3, 4)))
    hdf5_file.create_dataset("data_binarni", data=data_binary)


In [ ]:
with h5py.File("test_hdf5.h5", "r") as hdf5_file:
    data_hdf5 = hdf5_file["data1"]
    print(data_hdf5)
    # musíme data "nahrát" pomocí [:], jinak by byl výsledek jen "ukazatel" na data
    data_pole = hdf5_file["data1"][:]
    data_binary = hdf5_file["data_binarni"][:]
    data_nahodne = hdf5_file["data_nahodne"][:]



In [ ]:
# tato buňka záměrně vyvolá chybu: dataset už po zavření souboru není platný
# pokus o čtení přes uzavřený odkaz selže
data_hdf5[:]


V HDF5 může být velké množství dat, proto se často pracuje "lazy" stylem: dokud data skutečně nepotřebujeme v paměti (například jako NumPy pole), držíme jen odkaz na data v souboru.
